In [5]:
import requests
from xml.etree import ElementTree
import pandas as pd
from datetime import datetime

# Set pandas options to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Base URL for PubMed E-utilities API
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'

# Current date and January 1 of five years ago
current_date = datetime.now().strftime('%Y/%m/%d')
five_years_ago_date = datetime(datetime.now().year - 5, 1, 1).strftime('%Y/%m/%d')

# Your query
query = f'selective kinase inhibitor AND ("{five_years_ago_date}"[PDAT] : "{current_date}"[PDAT])'

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['Title', 'URL', 'Authors', 'Journal', 'Publication Year'])

# Send a GET request to the ESearch utility to search for the query in PubMed
response = requests.get(f'{base_url}esearch.fcgi', params={
    'db': 'pubmed',
    'term': query,
    'retmode': 'xml'
})

# If the request is successful, parse the XML response
if response.status_code == 200:
    root = ElementTree.fromstring(response.text)
    # Get the list of PubMed IDs (PMIDs) from the response
    id_list = [id_elem.text for id_elem in root.iter('Id')]

    # Create a list to store the data
    data_to_append = []

    # For each PMID, send a GET request to the ESummary utility to retrieve the article details
    for pmid in id_list:
        response = requests.get(f'{base_url}esummary.fcgi', params={
            'db': 'pubmed',
            'id': pmid,
            'retmode': 'xml'
        })
        if response.status_code == 200:
            root = ElementTree.fromstring(response.text)
            # Extract the title, URL, authors, journal, and publication year for each article and add them to the list
            for docsum in root.iter('DocSum'):
                title = docsum.find('Item[@Name="Title"]').text
                url = f'https://pubmed.ncbi.nlm.nih.gov/{pmid}/'
                authors = docsum.find('Item[@Name="Authors"]').text if docsum.find('Item[@Name="Authors"]') is not None else 'N/A'
                journal = docsum.find('Item[@Name="FullJournalName"]').text if docsum.find('Item[@Name="FullJournalName"]') is not None else 'N/A'
                pub_year = docsum.find('Item[@Name="PubDate"]').text.split()[0] if docsum.find('Item[@Name="PubDate"]') is not None else 'N/A'
                data_to_append.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year})

    # Concatenate the list of dictionaries to the DataFrame
    df = pd.concat([df, pd.DataFrame(data_to_append)], ignore_index=True)

# Display the DataFrame
print(df)

# Save the DataFrame as a CSV file
df.to_csv('literature.csv', index=False)


                                                Title  \
0   Genomic Discovery and Structure-Activity Explo...   
1   Repurposing Antiviral Drugs as Potential Anti-...   
2   Dapagliflozin restores autophagy and attenuate...   
3   Dual Adjuvant-Loaded Peptide Antigen Self-Asse...   
4   Statins enhance extracellular release of hepat...   
5   Co-Expression Network Analysis and Molecular D...   
6   A novel chemical genetic approach reveals para...   
7   Rhabdomyolysis Associated With Febuxostat in a...   
8   A novel disposable ultrasensitive sensor based...   
9   Immunohistochemical Expression of Caspase1 and...   
10  Guselkumab: a new etiological factor of Medica...   
11  Synthesis and evaluation of chemical linchpins...   
12  Exploring the therapeutic potential of a novel...   
13  Advances with janus kinase inhibitors for the ...   
14  Empagliflozin demonstrates cytotoxicity and sy...   
15  SARS-CoV-2 E protein interacts with BRD2 and B...   
16  Upfront versus deferred cyt

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
